In [1]:
print("Importing numpy.")
import numpy as np

print("Importing matplotlib.")
import matplotlib
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
%matplotlib widget
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Times New Roman']


print("Importing ipy.")
from ipywidgets import interact, widgets, FloatSlider, Dropdown, Checkbox, VBox
from IPython.display import display, clear_output

print("Importing PIL.")
from PIL import Image

print("Importing pandas.")
from pandas import DataFrame

import warnings
warnings.filterwarnings("ignore")

print("Importing keras.")
from keras.models import model_from_json

from tqdm import tqdm

from uproot3_methods import TLorentzVectorArray

Importing numpy.
Importing matplotlib.
Importing ipy.
Importing PIL.
Importing pandas.
Importing keras.


In [24]:
## LOAD MODEL
model_dir = f'../models/classifier/reco/model/'
json_file = open(model_dir + f'model_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights(model_dir + f'model_1.h5')

## LOAD INPUTS
file = np.load("../inputs/reco/nn_input_MX700_MY400_classifier.npz")
nevents = len(np.concatenate((file['test'], file['train'], file['val'])))

In [25]:
x_test = file['x_test']
X_test = file['X_test']
ntest = len(file['test'])

scores = model.predict(x_test)
scores = scores[:,0]
rscores = np.around(scores, decimals=3)

## APPLY MODEL TO INPUTS
pairings = []
score_dist = []
for i in range(15):
    pairings.append(scores[np.arange(i, ntest, 15)])

events = []
for i in range(0, ntest*15, 15):
    events.append(scores[i:i+15])

In [26]:
sbe = scores.reshape(5039,15) # scores by event

def get_passing_pairs(cut):
    """Returns how many pairs pass the cut for each event, as well as indices of passing pairs."""
    pass_pairs = np.array(())
    n = np.array(())
    higgs = np.array((), dtype=bool)
    for evt in sbe:
        evt_pass = [i for i,s in enumerate(evt) if s > cut]
        pass_pairs =  np.append(pass_pairs, evt_pass)
        n  = np.append(n, len(evt_pass))
        higgs  = np.append(higgs, evt[0] > cut and evt[1] > cut and evt[2] > cut)
    return pass_pairs, higgs, n

In [27]:
thresholds = np.arange(0.0,1.01,0.01)
passing_pairs = []
higgs_passing = []
num_passing = []
for cut in tqdm(thresholds):
    pass_pairs, higgs, n = get_passing_pairs(cut)
    passing_pairs.append(pass_pairs)
    num_passing.append(n)
    higgs_passing.append(higgs)

100%|██████████| 101/101 [00:30<00:00,  3.29it/s]


In [28]:
n_cut = []
for cut in tqdm(thresholds):
    n = [np.array(()) for _ in range(16)]
    for evt in sbe:
        n_evt = np.sum(evt > cut)
        sorted_evt = np.append(np.sort(evt[:3])[::-1], np.sort(evt[3:])[::-1])
        n[n_evt] = np.append(n[n_evt], sorted_evt)
    n_cut.append(n)

100%|██████████| 101/101 [00:14<00:00,  7.19it/s]


In [29]:
fig, axs = plt.subplots(nrows=6, ncols=3, figsize=(10,10))
n_cut[50]
for i,ax in enumerate(axs.flat):
    ax.set_title(f'{i} passing pairs', fontsize=12)
    if i >= 16: continue
    if len(n_cut[50]) > 0:
        ax.hist(n_cut[50][i], bins=thresholds)
        
        
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
x = thresholds
y = []
for n in np.arange(16):
    y_pair = np.array(())
    for i,cut in enumerate(thresholds):
        # Select only events with number of passing pairs less than or equal to n
        max_pair_mask = num_passing[i] <= n
        y_pair = np.append(y_pair, np.sum(higgs_passing[i][max_pair_mask])/np.sum(max_pair_mask))
    y.append(y_pair)

In [31]:
fig, ax = plt.subplots(figsize=(8,5))
for i,y_pair in enumerate(y):
    if i < 3: continue
    ax.plot(x, y_pair, label=fr'$\leq${i} passing pairs')
ax.set_xlabel('Classifier Score Cut')
ax.set_ylabel('Ratio of events with three passing Higgs')
ax.set_ylim(0,1)
ax.legend(fontsize=8, bbox_to_anchor=(1.05, 1), loc='upper left')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
H_pairs = [0, 1, 2]

def get_ratio(pair, threshold):
    return np.sum(scores[pair] > cut)/len(scores[pair])

empty =  []
d = {i:empty for i in range(15)}
d['3H'] = []


for cut in thresholds:
    for i in range(15):
        pair = np.arange(i,len(scores),15)
        eff = get_ratio(pair, cut)
        d[i] = d[i] + [eff]

    X_mask = np.arange(0,len(scores),15)
    Y1_mask = np.arange(1,len(scores),15)
    Y2_mask = np.arange(2,len(scores),15)
    mask = X_mask & Y1_mask & Y2_mask
    d['3H'] = d['3H'] +  [get_ratio(mask, cut)]


In [33]:
indices = np.array((), int)
for ind in np.argwhere(rscores == 0):
    indices = np.append(indices, ind)

In [34]:
label = [r'$H_X$', r'$H_{Y,1}$', r'$H_{Y,2}$', r'$b_{X,1}$, $b_{Y_1,1}$', r'$b_{X,1}$, $b_{Y_1,2}$', r'$b_{X,1}$, $b_{Y_2,1}$', r'$b_{X,1}$, $b_{Y_2,2}$', r'$b_{X,2}$, $b_{Y_1,1}$', r'$b_{X,2}$, $b_{Y_1,2}$', r'$b_{X,2}$, $b_{Y_2,1}$', r'$b_{X,2}$, $b_{Y_2,2}$', r'$b_{Y_1,1}$, $b_{Y_2,1}$', r'$b_{Y_1,1}$, $b_{Y_2,2}$', r'$b_{Y_1,2}$, $b_{Y_2,1}$', r'$b_{Y_1,2}$, $b_{Y_2,2}$']
di = {}
di[r'$3H$'] = d['3H']
for i in np.arange(0,15):
    di[label[i]] = d[i]

thresholds = np.round(thresholds, 2)
df = DataFrame.from_dict(data=di, orient='index', columns=thresholds).round(decimals=3)
df

,0.00,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,...,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99,1.00
$3H$,1.0,0.844,0.805,0.781,0.767,0.750,0.738,0.724,0.713,0.704,...,0.319,0.308,0.296,0.281,0.262,0.242,0.217,0.176,0.002,0.0
$H_X$,1.0,0.992,0.990,0.988,0.987,0.987,0.985,0.983,0.981,0.979,...,0.712,0.695,0.672,0.649,0.616,0.578,0.526,0.441,0.006,0.0
"$H_{Y,1}$",1.0,0.993,0.990,0.988,0.987,0.986,0.985,0.985,0.984,0.983,...,0.781,0.765,0.748,0.726,0.704,0.666,0.611,0.514,0.003,0.0
"$H_{Y,2}$",1.0,0.983,0.977,0.972,0.968,0.966,0.962,0.958,0.955,0.953,...,0.369,0.347,0.325,0.297,0.267,0.228,0.186,0.135,0.002,0.0
"$b_{X,1}$, $b_{Y_1,1}$",1.0,0.179,0.145,0.130,0.121,0.116,0.112,0.108,0.103,0.098,...,0.028,0.027,0.026,0.025,0.023,0.022,0.020,0.018,0.009,0.0
"$b_{X,1}$, $b_{Y_1,2}$",1.0,0.562,0.501,0.460,0.437,0.415,0.400,0.387,0.374,0.366,...,0.110,0.106,0.100,0.093,0.089,0.080,0.069,0.054,0.003,0.0
"$b_{X,1}$, $b_{Y_2,1}$",1.0,0.589,0.544,0.519,0.498,0.478,0.465,0.454,0.446,0.440,...,0.192,0.185,0.177,0.169,0.158,0.142,0.124,0.098,0.003,0.0
"$b_{X,1}$, $b_{Y_2,2}$",1.0,0.850,0.800,0.767,0.743,0.722,0.702,0.685,0.672,0.660,...,0.253,0.241,0.229,0.213,0.193,0.173,0.149,0.117,0.003,0.0
"$b_{X,2}$, $b_{Y_1,1}$",1.0,0.221,0.186,0.167,0.153,0.142,0.136,0.130,0.125,0.120,...,0.016,0.016,0.015,0.014,0.014,0.013,0.013,0.012,0.005,0.0
"$b_{X,2}$, $b_{Y_1,2}$",1.0,0.707,0.652,0.617,0.593,0.573,0.556,0.538,0.523,0.508,...,0.055,0.052,0.048,0.042,0.036,0.030,0.024,0.018,0.001,0.0


In [35]:
def get_bars(val):
    return [d[i][val] for i in np.arange(15)]

def update(val = 1):
    y = get_bars(val)
    
    ax.bar(x, y)
    fig.canvas.draw_idle()

In [36]:
x_dict = {'HX':0,
 'HY1':1,
 'HY2':2,
 'X b1, Y1 b1':3,
 'X b1, Y1 b2':4,
 'X b1, Y2 b1':5,
 'X b1, Y2 b2':6,
 'X b2, Y1 b1':7,
 'X b2, Y1 b2':8,
 'X b2, Y2 b1':9,
 'X b2, Y2 b2':10,
 'Y1 b1, Y2 b1':11,
 'Y1 b1, Y2 b2':12,
 'Y1 b2, Y2 b1':13,
 'Y1 b2, Y2 b2':14}

In [37]:
x_drop = ['HX',
 'HY1',
 'HY2',
 'X b1, Y1 b1',
 'X b1, Y1 b2',
 'X b1, Y2 b1',
 'X b1, Y2 b2',
 'X b2, Y1 b1',
 'X b2, Y1 b2',
 'X b2, Y2 b1',
 'X b2, Y2 b2',
 'Y1 b1, Y2 b1',
 'Y1 b1, Y2 b2',
 'Y1 b2, Y2 b1',
 'Y1 b2, Y2 b2']

In [38]:
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection

In [39]:
def dropdown_event(change):
    global a
    a.remove()
    pos = x_dict[change.new] + 1
    arrow = mpatches.Arrow(pos, 1.2, 0, -0.2, color='blue')
    a = bar_ax.add_patch(arrow)
    
    X_bbar .set_visible(False)
    X_b.set_visible(False)
    Y1_bbar.set_visible(False)
    Y1_b.set_visible(False)
    Y2_bbar.set_visible(False)
    Y2_b.set_visible(False)
    
    if 'b' in change.new:
        part1, part2 = change.new.split(',')
        if ('X' in part1 and 'b1' in part1) or ('X' in part2 and 'b1' in part2):
            X_b.set_visible(True)
        if ('X' in part1 and 'b2' in part1) or ('X' in part2 and 'b2' in part2):
            X_bbar.set_visible(True)
        if ('Y1' in part1 and 'b1' in part1) or ('Y1' in part2 and 'b1' in part2):
            Y1_b.set_visible(True)
        if ('Y1' in part1 and 'b2' in part1) or ('Y1' in part2 and 'b2' in part2):
            Y1_bbar.set_visible(True)
        if ('Y2' in part1 and 'b1' in part1) or ('Y2' in part2 and 'b1' in part2):
            Y2_b.set_visible(True)
        if ('Y2' in part1 and 'b2' in part1) or ('Y2' in part2 and 'b2' in part2):
            Y2_bbar.set_visible(True)
    elif 'H' in change.new:
        if 'X' in change.new:
            X_b.set_visible(True)
            X_bbar.set_visible(True)
        if 'Y1' in change.new:
            Y1_b.set_visible(True)
            Y1_bbar.set_visible(True)
        if 'Y2' in change.new:
            Y2_b.set_visible(True)
            Y2_bbar.set_visible(True)
            
def f(threshold):
    global fig, bar_ax
    bar_ax.clear()
    bar_ax.set_ylim(0,1.1)
    bar_ax.set_xlim(-1,16)
    bar_ax.set_ylabel('Ratio of events above threshold', fontsize=18)
    bar_ax.set_xlabel('Pairs', fontsize=18)
    bar_ax.set_xticklabels(labels=[r'$3H$'] + label, rotation=45,  ha='right')
    y = get_bars(int(threshold*100))
    Higgs = [d['3H'][int(threshold*100)]] + [y[0], y[1], y[2]] + [0]*12
    Non_Higgs = [0]*4 + y[3:]
    Higgs_bars = bar_ax.bar(x, Higgs, color='C2')
    nonHiggs_bars = bar_ax.bar(x, Non_Higgs, color='C3')
    fig.canvas.draw() 
    renderer = fig.canvas.renderer 
    bar_ax.draw(renderer)
    plt.tight_layout()
    plt.show()

In [41]:
threshold = 0.5
fig = plt.figure(figsize=(12,6))
gs = GridSpec(1, 2, width_ratios=[6, 5]) 
bar_ax = fig.add_subplot(gs[0])
x = [r'$3H$'] + label
y = get_bars(int(threshold*100))
Higgs = [d['3H'][int(threshold*100)]] + [y[0], y[1], y[2]] + [0]*12
Non_Higgs = [0]*4 + y[3:]
Higgs_bars = bar_ax.bar(x, Higgs, color='C2')
nonHiggs_bars = bar_ax.bar(x, Non_Higgs, color='C3')
bar_ax.set_ylim(0,1.1)
bar_ax.set_xlim(-1,16)
bar_ax.set_ylabel('Ratio of events above threshold', fontsize=18)
bar_ax.set_xlabel('Pairs', fontsize=18)
bar_ax.set_xticklabels(labels=[r'$3H$'] + label, rotation=45,  ha='right')

arrow = mpatches.Arrow(1, 1.2, 0, -0.2, color='blue')
a = bar_ax.add_patch(arrow)

ax = fig.add_subplot(gs[1], frameon=False)
ax.axis('off')
ax.set_ylim(0,1)
ax.set_xlim(0,1)

X_bbar  = mpatches.Circle((0.770, 0.150), 0.04, color='blue', fill=False, zorder=10)
X_b     = mpatches.Circle((0.770, 0.320), 0.04, color='blue', fill=False, zorder=10)
Y1_bbar = mpatches.Circle((0.945, 0.375), 0.04, color='blue', fill=False, zorder=10)
Y1_b    = mpatches.Circle((0.945, 0.545), 0.04, color='blue', fill=False, zorder=10)
Y2_bbar = mpatches.Circle((0.945, 0.675), 0.04, color='blue', fill=False, zorder=10)
Y2_b    = mpatches.Circle((0.945, 0.845), 0.04, color='blue', fill=False, zorder=10)

X_bbar.set_visible(True)
X_b.set_visible(True)
Y1_bbar.set_visible(False)
Y1_b.set_visible(False)
Y2_bbar.set_visible(False)
Y2_b.set_visible(False)

ax.add_patch(X_bbar)
ax.add_patch(X_b)
ax.add_patch(Y1_bbar)
ax.add_patch(Y1_b)
ax.add_patch(Y2_bbar)
ax.add_patch(Y2_b)

image = Image.open('../images/CascadeDecay.png')
im = OffsetImage(image, zoom=0.35)
ab = AnnotationBbox(im, (0.5,0.5), xycoords='data', boxcoords="offset points",  pad=0.3,  arrowprops=dict(arrowstyle="->"))
ax.add_artist(ab)

pair = Dropdown(options = x_drop)
    
interact(f, threshold=FloatSlider(value=0.5, min=0, max=1.0, step=0.01, continuous_update=False))
pair.observe(dropdown_event, names='value')
display(pair)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='threshold', max=1.0, step=0…

Dropdown(options=('HX', 'HY1', 'HY2', 'X b1, Y1 b1', 'X b1, Y1 b2', 'X b1, Y2 b1', 'X b1, Y2 b2', 'X b2, Y1 b1…

In [20]:
from consistent_plots import hist

In [21]:
threshold = 0.5
bins = np.linspace(0,5,100)
fig, ax = plt.subplots(figsize=(8,5))
ax.set_xlim(0,5)
ax.set_ylim(0,400)
showing = np.array(([True]*15), dtype=bool)
colors = [f'C{i}' for i in range(15)]

for i in range(15):
    pair_mask = np.arange(i,ntest*15,15)
    data = X_test[pair_mask]
    sc = scores[pair_mask]
    # 6 for dR
    n, bins, patches = hist(ax, data[sc > threshold,6], label=x_drop[i], color=colors[i], bins=bins)
ax.legend()

def f(cut, c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14):
    global fig, ax, threshold, bins
    showing = [c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14]
    threshold = cut
    ax.clear()
    ax.set_xlim(0,5)
    ax.set_ylim(0,400)
    for i in range(15):
        if showing[i] == True:
            pair_mask = np.arange(i,ntest*15,15)
            data = X_test[pair_mask]
            sc = scores[pair_mask]
            # 6 for dR
            n, bins, patches = hist(ax, data[sc > threshold,6], label=x_drop[i], color=colors[i], bins=bins)
    ax.legend(np.array((x_drop))[showing])
    fig.canvas.draw() 
    renderer = fig.canvas.renderer 
    ax.draw(renderer)
    plt.tight_layout()
    plt.show()
    
slider = FloatSlider(value=0.5, min=0, max=1.0, step=0.01, continuous_update=False)
checkboxes = [widgets.ToggleButton(value=True, description=label) for label in x_drop]

i = widgets.interactive(f, 
                        cut=slider, 
                        c0=checkboxes[0],   c1=checkboxes[1],   c2=checkboxes[2], 
                        c3=checkboxes[3],   c4=checkboxes[4],   c5=checkboxes[5],
                        c6=checkboxes[6],   c7=checkboxes[7],   c8=checkboxes[8],
                        c9=checkboxes[9],   c10=checkboxes[10], c11=checkboxes[11],
                        c12=checkboxes[12], c13=checkboxes[13], c14=checkboxes[14])

hbox1 = widgets.HBox(checkboxes[:4])
hbox2 = widgets.HBox(checkboxes[4:8])
hbox3 = widgets.HBox(checkboxes[8:12])
hbox4 = widgets.HBox(checkboxes[12:])
vbox = VBox([slider, hbox1, hbox2, hbox3, hbox4])
vbox

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …